# Multi-Turn Conversation Dataset Generator

## Deskripsi
Notebook ini menghasilkan dataset percakapan multi-turn untuk melatih AI sebagai interviewer karier di platform Diploy.

## Setup Awal

### 1. Install Dependencies
```bash
pip install openai pandas openpyxl tqdm python-dotenv
```

### 2. Set API Key
**Untuk keamanan, JANGAN hardcode API key!**

#### Opsi A: Environment Variable (Recommended)
```bash
export OPENAI_API_KEY='your-api-key-here'
```

#### Opsi B: File .env
Buat file `.env` di root folder:
```
OPENAI_API_KEY=your-api-key-here
```
Lalu load dengan:
```python
from dotenv import load_dotenv
load_dotenv()
```

### 3. Persiapkan Data
Pastikan file Excel `datasetmultiturncreateV2.xlsx` memiliki kolom:
- Jenjang Pendidikan
- Jurusan
- Pelatihan
- Sertifikasi
- Pekerjaan Saat Ini
- Pengalaman Kerja
- Keterampilan
- Area_Fungsi
- Level

## Output Format
File JSONL dengan format:
```json
{
  "messages": [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."}
  ]
}
```

## Mode Percakapan
- **FAST_DIRECT**: Langsung rekomendasi
- **FAST_SHORT**: 1 pertanyaan → rekomendasi
- **MEDIUM**: 4-6 turn dengan 1-2 pertanyaan
- **LONG**: 8-10 turn dengan 3-5 pertanyaan

In [1]:
import os
from pathlib import Path

# =========================
# PATH CONFIG
# =========================
# Untuk Google Colab, uncomment baris berikut:
# from google.colab import drive
# drive.mount('/content/drive')
# DATASET_DIR = "/content/drive/MyDrive/Colab Notebooks"

DATASET_DIR = Path.cwd().parent / "Flagged_500_Per_Class/Rows_1-100"
DATASET_DIR = str(DATASET_DIR)

print(f"Dataset directory: {DATASET_DIR}")
print(f"Directory exists: {os.path.exists(DATASET_DIR)}")

# List files untuk verifikasi
if os.path.exists(DATASET_DIR):
    files = os.listdir(DATASET_DIR)
    excel_files = [f for f in files if f.endswith('.xlsx')]
    print(f"Found {len(excel_files)} Excel files")
    if excel_files:
        print(f"   First few: {excel_files[:3]}")
else:
    print("WARNING: Directory not found!")

Dataset directory: /Users/irz/Downloads/dtp-data-pipeline-main/Pipeline Multiturn/Flagged_500_Per_Class/Rows_1-100
Directory exists: True
Found 45 Excel files
   First few: ['Pengembangan_Produk_Digital_5.xlsx', 'Tata_Kelola_Teknologi_Informasi_9.xlsx', 'Teknologi_dan_Infrastruktur_6.xlsx']


## Quick Setup: Set API Key

**Pilih salah satu cara berikut:**

### Cara 1: Langsung di Notebook (Paling Cepat)
Jalankan cell ini terlebih dahulu, lalu jalankan cell berikutnya:
```python
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-your-actual-key-here'  # Ganti dengan key Anda
```

### Cara 2: Buat File .env
Buat file `.env` di folder `/home/wildanaziz/dtp-data-pipeline/Pipeline Multiturn/script/` dengan isi:
```
OPENAI_API_KEY=sk-proj-your-actual-key-here
```

### Cara 3: Terminal (Persistent)
```bash
export OPENAI_API_KEY='sk-proj-your-actual-key-here'
```
Lalu restart kernel notebook.

## ⚠️ PENTING: Restart Kernel

**Jika Anda mendapat error ValueError pada format string:**

1. **Restart Kernel**: Klik menu `Kernel` → `Restart Kernel` atau tekan tombol restart di toolbar
2. **Jalankan ulang cell 2** (PATH CONFIG)
3. **Jalankan ulang cell 6** (Main processing code)

**Alasan**: File sudah diperbaiki, tapi kernel masih menyimpan kode lama di memori.

In [ ]:
import pandas as pd
import json
import asyncio
import random
from openai import AsyncOpenAI, APIError, RateLimitError
import time
import os
from tqdm.auto import tqdm
from datetime import datetime
from pathlib import Path
import glob
import re

# load env var
try:
    from dotenv import load_dotenv
    load_dotenv()  # Load dari .env file
    print(".env file loaded successfully")
except ImportError:
    print("python-dotenv tidak terinstall, gunakan environment variable manual")
except Exception as e:
    print(f"Error loading .env: {e}")

# config env
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

if not OPENAI_API_KEY:
    env_file = Path.cwd() / ".env"
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY'):
                    OPENAI_API_KEY = line.split('=')[1].strip().strip('"').strip("'")
                    print("API Key loaded from .env file manually")
                    break
    
    if not OPENAI_API_KEY:
        print("\n" + "="*60)
        print("OPENAI_API_KEY TIDAK DITEMUKAN!")
        print("="*60)
        print("\nCara Set API Key:")
        print("\nOpsi 1: Buat file .env di folder ini:")
        print(f"   Location: {Path.cwd()}")
        print("   Isi file .env:")
        print('   OPENAI_API_KEY=sk-proj-your-actual-key-here')
        print("\nOpsi 2: Set di notebook (temporary):")
        print("   Tambahkan cell baru dengan:")
        print("   import os")
        print("   os.environ['OPENAI_API_KEY'] = 'sk-proj-your-actual-key-here'")
        print("\nOpsi 3: Set di terminal (persistent):")
        print("   export OPENAI_API_KEY='sk-proj-your-actual-key-here'")
        print("   Lalu restart kernel notebook")
        print("\n" + "="*60 + "\n")
        raise ValueError("OPENAI_API_KEY tidak ditemukan! Ikuti instruksi di atas.")

print(f"API Key: {OPENAI_API_KEY[:20]}... (truncated for security)")

client = AsyncOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# atur batch size konsisten
BATCH_SIZE = 25
MAX_TOKENS = 1400
TEMPERATURE = 0.7

RETRY_LIMIT = 3
RETRY_DELAY = 3  
CONCURRENT_REQUESTS = 1

# pilih model list yang ada di openrouter pastiin pake yg gpt aje
MODEL_NAME = "google/gemini-2.5-flash"  

# output dir config
OUTPUT_BASE_DIR = Path.cwd().parent / "MultiturnDatasetOutput_1-100"  

print(f"Output akan disimpan di: {OUTPUT_BASE_DIR}")

# SYSTEM PROMPT (LOCKED) — versi baru sesuai desain percakapan Diploy
SYSTEM_PROMPT = (
    "Anda adalah interviewer dari platform talenta digital Diploy khusus Area Fungsi. Tugas Anda adalah menggali detail kompetensi talenta berdasarkan data awal yang diberikan, meluruskan jawaban yang kurang relevan, dan memastikan informasi yang terkumpul cukup tajam untuk pemetaan Area Fungsi dan Level Okupasi. Gunakan bahasa Indonesia yang baik dan benar, tetap profesional, dan jangan menggunakan bahasa gaul atau singkatan informal."
)

# GLOBAL RULES – format percakapan & output JSON yang baru
GLOBAL_RULES = """
ATURAN FORMAT OUTPUT
====================
1. Output WAJIB berupa ARRAY JSON VALID yang berisi daftar pesan percakapan.
2. Setiap elemen array WAJIB memiliki:
   - "role": hanya boleh "user" atau "assistant"
   - "content": string isi pesan.
3. TIDAK BOLEH ada objek lain di luar array tersebut dan TIDAK BOLEH ada teks di luar JSON.

STRUKTUR WAJIB
==============
1) Pesan pertama:
   - role = "user"
   - pesan pertama selalu dari user. content berisi ringkasan profil dari data yang diberikan dengan format:
     "Berikut data singkat saya:\\n<Field 1>: <isi>\\n<Field 2>: <isi>\\n..."
   - Informasi yang digunakan HANYA boleh diambil dari bagian PROFILE DATA (tidak boleh diubah).

2) Pesan-pesan berikutnya:
   - Bergantian antara "assistant" dan "user" sesuai kebutuhan percakapan.
   - Setelah pesan pertama dari user, assistant mengawali pembicaraan dengan contoh seperti ini: "Halo, terima kasih sudah menyempatkan waktu untuk mengikuti interview ini. Saya akan mengajukan beberapa pertanyaan terkait kompetensi Anda. (diteruskan dengan pertanyaan pertama)"
   - Assistant:
     * hanya boleh menanyakan hal yang relevan dengan pendidikan, tugas akhir, pelatihan, sertifikasi, pengalaman kerja, dan keterampilan yang ada di data.
     * WAJIB menggali lebih dalam jika jawaban user terlalu singkat atau masih umum.
     * WAJIB meluruskan jika jawaban user keluar konteks, lalu mengembalikan fokus ke pertanyaan.

3) Pesan terakhir:
   - role = "assistant"
   - content WAJIB diawali dengan tag persis: "[END OF CHAT]".
   - Setelah tag tersebut:
     * ucapkan terima kasih karena sudah melakukan interview,
     * sebutkan secara eksplisit Area Fungsi dan Level yang sudah dipetakan (lihat TEMPLATE RINGKASAN AKHIR),
     * tambahkan 1–3 kalimat alasan singkat dan to the point, berdasarkan:
       - latar belakang pendidikan,
       - pengalaman kerja,
       - pelatihan/sertifikasi,
       - dan keterampilan yang muncul di data dan jawaban user.
   - TIDAK BOLEH ada pertanyaan baru setelah bagian ringkasan akhir.

GAYA BAHASA
===========
- Assistant:
  * menggunakan bahasa Indonesia yang baku, jelas, dan profesional,
  * menggunakan kata ganti "Anda",
  * TIDAK menggunakan singkatan informal seperti "yg", "gk", "ga", "dr", "tp", "utk", "sdh", "blm", dsb.
- User:
  * secara umum boleh menjawab dengan bahasa yang natural,
  * sesekali boleh muncul singkatan ringan, tetapi tetap sopan dan mudah dipahami.

HAL YANG DILARANG
=================
- Menginvensi sertifikasi, pelatihan, pekerjaan, atau keterampilan baru yang tidak ada di data.
- Menulis JSON di dalam string "content".
- Mengubah target Area Fungsi dan Level yang sudah diberikan di prompt.
"""

# MODE RULES - Deskripsi eksplisit dalam bahasa Indonesia
MODE_RULES = {
    "fast_direct": "PERCAKAPAN LANGSUNG TANPA PERTANYAAN: Tidak ada pertanyaan interview. User memberikan data profil, lalu assistant langsung memberikan kesimpulan rekomendasi Area Fungsi dan Level beserta alasannya. Total 2 pesan (1 user, 1 assistant dengan [END OF CHAT]).",
    
    "fast_short": "PERCAKAPAN SINGKAT: Assistant mengajukan 1-2 pertanyaan singkat untuk konfirmasi atau klarifikasi data penting. Total 2-5 pesan bergantian, diakhiri assistant dengan [END OF CHAT].",
    
    "medium": "PERCAKAPAN SEDANG: Assistant melakukan interview dengan 3-5 pertanyaan untuk menggali kompetensi lebih dalam terkait pengalaman kerja, pelatihan, dan keterampilan. Total 6-11 pesan bergantian, diakhiri assistant dengan [END OF CHAT].",
    
    "long": "PERCAKAPAN PANJANG: Assistant melakukan interview mendalam dengan 6-8 pertanyaan detail mengenai pendidikan, tugas akhir, pelatihan, sertifikasi, pengalaman kerja, tanggung jawab, dan keterampilan. Jika user menjawab kurang detail, assistant menggali lebih dalam. Total 12-15 pesan bergantian, diakhiri assistant dengan [END OF CHAT]."
}

FAST_VARIANTS = ["fast_direct", "fast_short"]

# Normalisasi
MISSING = {"", "-", "–", "—", "none", "nan", "n/a", "null", "tidak ada"}

def normalize(v):
    """Normalisasi nilai kosong/missing."""
    if v is None or pd.isna(v):
        return None
    s = str(v).strip()
    return None if s.lower() in MISSING else s

# Extract fields
def extract_fields(row):
    """Ekstrak field public (untuk conversation) dan hidden (untuk validasi)."""
    pub_map = {
        "Jenjang_Pendidikan": "Jenjang Pendidikan",  
        "Jurusan": "Jurusan",
        "Judul_Tugas_Akhir": "Judul Tugas Akhir",
        "Bidang_Pelatihan": "Bidang Pelatihan",  
        "Nama_Pelatihan": "Nama Pelatihan",  
        "Sertifikasi": "Sertifikasi",
        "Bidang_Sertifikasi": "Bidang Sertifikasi",
        "Posisi_Pekerjaan": "Posisi Pekerjaan",  
        "Deskripsi_tugas_dan_tanggung_jawab": "Deskripsi Tugas dan Tanggung Jawab",  
        "Lama_Bekerja": "Lama Bekerja",  
        "Keterampilan": "Keterampilan",
    }

    hid_map = {
        "Area_Fungsi": "Area Fungsi", 
        "Level_Okupasi": "Level"
    }

    public, hidden = {}, {}

    for c, k in pub_map.items():
        v = normalize(row.get(c))
        if v:
            public[k] = v

    for c, k in hid_map.items():
        v = normalize(row.get(c))
        if v:
            if k == "Level":
                try:
                    v = str(int(float(v)))
                except Exception:
                    v = v
            hidden[k] = v

    return public, hidden

# Build user intro
def make_user_intro(public):
    """Generate user introduction dari data profil - mencakup SEMUA kolom."""
    if not public:
        return "Halo, saya ingin mengikuti asesmen karier."
    
    # Format: "Berikut data singkat saya:\n<Field>: <Value>\n..."
    lines = ["Halo, saya ingin mengikuti asesmen karier. Berikut data singkat saya:"]
    
    for field, value in public.items():
        lines.append(f"{field}: {value}")
    
    return "\n".join(lines)

# Build prompt
def build_prompt(row, mode_key):
    """Generate prompt untuk GPT model sesuai format baru Diploy."""
    public, hidden = extract_fields(row)

    # PROFILE DATA untuk referensi model
    pdesc = "\n".join([f"- {k}: {v}" for k, v in public.items()])

    # Ambil level & area fungsi dari hidden
    level_val = hidden.get("Level", "-")
    try:
        level_val = str(int(float(level_val)))
    except Exception:
        pass

    area_val = hidden.get("Area Fungsi", "")

    return f"""
PROFILE DATA (REFERENSI, JANGAN DIUBAH APA ADANYA):
{pdesc}

KONTEKS:
- Anda akan mensimulasikan percakapan antara user dan interviewer di platform digital talent pool
- Area Fungsi dan Level akan dipetakan berdasarkan data kualifikasi talenta dan hasil jawaban interview. namun dari excel ini sudah ada Area Fungsi dan Level nya untuk anda mengetahui konteks simulasi pembicaraan yang akan dibuat. Data Area Fungsi dan Level TIDAK BOLEH diubah:
  * Area Fungsi: {area_val}
  * Level Okupasi: {level_val}

MODE PERCAPAKAN:
{MODE_RULES[mode_key]}

ATURAN FORMAT & PERILAKU:
{GLOBAL_RULES}

INSTRUKSI KHUSUS:
- Pesan pertama (role="user") HARUS berupa ringkasan profil dengan pola:
  "Berikut data singkat saya:\\n<Field 1>: <isi>\\n<Field 2>: <isi>\\n..."
  dan ISINYA wajib konsisten dengan PROFILE DATA di atas. Tidak boleh mengubah data
- Anda bebas mengatur panjang percakapan dan jumlah turn (pertanyaan interview dan balasan) sesuai MODE, tetapi WAJIB menjaga fokus pada data yang dimiliki user dan menggali informasi di user hingga dirasa informasi yang didapat sudah cukup untuk memetakan user ke Area Fungsi dan Level tertentu.

TEMPLATE RINGKASAN AKHIR (WAJIB DIPAKAI DI TURN TERAKHIR ASSISTANT, SILAKAN SESUAIKAN BAGIAN ALASAN):
"[END OF CHAT] Terima kasih telah melakukan interview. Setelah saya pertimbangkan dari data yang Anda miliki dan interview yang sudah kita lakukan, Anda mendapat Area Fungsi {area_val} dan Level {level_val}. Anda memiliki kompetensi ... (penjelasan singkat alasan rea) <RESULT>{{\"area_fungsi\":\"{area_val}\", \"level\":{level_val}}}</RESULT>"

TUGAS ANDA:
- Hasilkan SATU ARRAY JSON VALID yang berisi daftar pesan percakapan.
- Setiap elemen array mempunyai "role" dan "content".
- System prompt TIDAK BOLEH dimasukkan ke dalam array (system prompt sudah diberikan terpisah).
"""

# enhanced validation function
def validate_conversation_structure(messages):
    """
    Validasi struktur conversation untuk memastikan format yang benar
    sesuai desain SFT Diploy (tanpa system di dalam array).
    
    Returns:
        tuple: (is_valid, error_message)
    """
    if not isinstance(messages, list):
        return False, "Messages bukan list"
    
    if len(messages) < 2:
        return False, f"Conversation terlalu pendek: {len(messages)} messages"

    # Pesan pertama harus user
    first = messages[0]
    if first.get("role") != "user":
        return False, f"Pesan pertama bukan 'user' tetapi {first.get('role')}"

    # Pesan terakhir harus assistant
    last = messages[-1]
    if last.get("role") != "assistant":
        return False, f"Pesan terakhir bukan 'assistant' tetapi {last.get('role')}"

    # Pesan terakhir harus diawali [END OF CHAT]
    last_content = last.get("content", "")
    if not isinstance(last_content, str) or not last_content.strip().startswith("[END OF CHAT]"):
        return False, "Pesan terakhir assistant wajib diawali '[END OF CHAT]' dan diakhiri <RESULT>{\"area_fungsi\":\"{area_val}\", \"level\":{level_val}}</RESULT>"

    # Cek korupsi JSON di dalam string
    for i, msg in enumerate(messages):
        content = msg.get("content", "")
        if isinstance(content, str):
            stripped = content.strip()
            if stripped.startswith("[{") or stripped.startswith("[\n  {"):
                return False, f"Message {i} ({msg.get('role')}) contains JSON array as string (corruption detected)"
            if '\\"role\\"' in content or '\\\"role\\\"' in content:
                return False, f"Message {i} ({msg.get('role')}) contains escaped JSON (corruption detected)"

    # Cek field wajib & role valid
    for i, msg in enumerate(messages):
        if "role" not in msg:
            return False, f"Message {i} missing 'role' field"
        if "content" not in msg:
            return False, f"Message {i} missing 'content' field"
        if msg["role"] not in ("user", "assistant"):
            return False, f"Message {i} has invalid role: {msg['role']}"

    return True, "Valid"

def clean_and_parse_json(raw_text):
    """
    Advanced JSON cleaning and parsing dengan multiple strategies.
    
    Returns:
        tuple: (parsed_array, success, error_message)
    """
    # Stage 1: Basic cleaning
    cleaned = (
        raw_text.replace("```json", "")
                .replace("```", "")
                .replace("\n\n", "\n")
                .strip()
    )
    
    # Stage 2: Remove common prefixes
    prefixes = [
        "Here is the JSON array:",
        "Berikut adalah array JSON:",
        "JSON output:",
        "Output JSON:",
        "Here's the conversation:",
        "Berikut percakapannya:",
    ]
    for prefix in prefixes:
        if cleaned.startswith(prefix):
            cleaned = cleaned[len(prefix):].strip()
    
    # Stage 3: Direct parse attempt
    if cleaned.startswith("[") and cleaned.endswith("]"):
        try:
            arr = json.loads(cleaned)
            return arr, True, None
        except json.JSONDecodeError as e:
            # Try repair
            try:
                repaired = re.sub(r',\s*}', '}', cleaned)
                repaired = re.sub(r',\s*]', ']', repaired)
                arr = json.loads(repaired)
                return arr, True, None
            except:
                pass 
    
    # Stage 4: Extract array substring
    if "[" in cleaned and "]" in cleaned:
        try:
            start = cleaned.index("[")
            end = cleaned.rindex("]") + 1
            extracted = cleaned[start:end]
            
            arr = json.loads(extracted)
            return arr, True, None
        except json.JSONDecodeError as e:
            # Try repair on extracted
            try:
                repaired = re.sub(r',\s*}', '}', extracted)
                repaired = re.sub(r',\s*]', ']', repaired)
                arr = json.loads(repaired)
                return arr, True, None
            except:
                return None, False, f"JSON parse error: {str(e)}"
    
    return None, False, "No valid JSON array found in response"

# OpenAI call with ENHANCED error handling and validation
async def call_api(prompt, row_index=None, mode="unknown"):
    """
    Call OpenAI API dengan retry mechanism, validation, dan error handling.
    Versi ini:
    - Mengharapkan output berupa ARRAY JSON pesan tanpa system.
    - Hanya mengizinkan role 'user' dan 'assistant' di dalam array.
    - Memastikan pesan terakhir diawali '[END OF CHAT]'.
    """
    for attempt in range(RETRY_LIMIT):
        try:
            # Adjust temperature based on attempt (lower = more deterministic)
            attempt_temp = max(0.3, TEMPERATURE - (attempt * 0.1))
            
            resp = await client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=MAX_TOKENS,
                temperature=attempt_temp,
            )

            raw = resp.choices[0].message.content.strip()

            # Parse with enhanced cleaning
            parsed_arr, parse_success, parse_error = clean_and_parse_json(raw)
            
            if not parse_success:
                print(f"Parse failed (row {row_index}, {mode}, attempt {attempt+1}): {parse_error}")
                
                # Retry dengan parameter berbeda
                if attempt < RETRY_LIMIT - 1:
                    await asyncio.sleep(RETRY_DELAY)
                    continue
                else:
                    # Fallback: kembalikan percakapan minimal dengan END OF CHAT
                    return [
                        {
                            "role": "user",
                            "content": "Berikut data singkat saya:\n(terjadi kegagalan parsing output percakapan)."
                        },
                        {
                            "role": "assistant",
                            "content": '[END OF CHAT] Terima kasih telah melakukan interview. Namun terjadi kesalahan teknis saat memproses percakapan. <RESULT>{"area_fungsi":"unknown", "level":null}</RESULT>'
                        },
                    ]
            
            # Bersihkan & filter hanya role user/assistant
            cleaned_arr = []
            for m in parsed_arr:
                if not isinstance(m, dict):
                    continue
                role = m.get("role")
                content = m.get("content")
                if role not in ("user", "assistant"):
                    # Buang system / role lain jika ada
                    continue
                cleaned_arr.append({"role": role, "content": content})
            
            # CRITICAL: Validate conversation structure
            is_valid, validation_error = validate_conversation_structure(cleaned_arr)
            
            if not is_valid:
                print(f"Validation failed (row {row_index}, {mode}, attempt {attempt+1}): {validation_error}")
                
                # Retry dengan parameter berbeda
                if attempt < RETRY_LIMIT - 1:
                    await asyncio.sleep(RETRY_DELAY)
                    continue
                else:
                    # Fallback: kembalikan percakapan minimal yang valid
                    return [
                        {
                            "role": "user",
                            "content": "Berikut data singkat saya:\n(percakapan tidak valid menurut aturan)."
                        },
                        {
                            "role": "assistant",
                            "content": '[END OF CHAT] Terima kasih telah melakukan interview. Namun percakapan yang dihasilkan tidak memenuhi aturan format. <RESULT>{"area_fungsi":"unknown", "level":null}</RESULT>'
                        },
                    ]
            
            # Sukses
            return cleaned_arr

        except (APIError, RateLimitError) as e:
            print(f"API error (row {row_index}, {mode}, attempt {attempt+1}/{RETRY_LIMIT}): {e}")
            await asyncio.sleep(RETRY_DELAY)
        except Exception as e:
            print(f"Unexpected error (row {row_index}, {mode}, attempt {attempt+1}): {e}")
            await asyncio.sleep(RETRY_DELAY)

    # All retries exhausted
    print(f"Row {row_index} ({mode}) FAILED after {RETRY_LIMIT} attempts")
    return [
        {
            "role": "user",
            "content": "Berikut data singkat saya:\n(semua percobaan pemanggilan API gagal)."
        },
        {
            "role": "assistant",
            "content": '[END OF CHAT] Terima kasih telah melakukan interview. Namun semua percobaan pemanggilan API gagal. <RESULT>{"area_fungsi":"unknown", "level":null}</RESULT>'
        },
    ]

# Batch processing with semaphore
SEM = asyncio.Semaphore(CONCURRENT_REQUESTS)

async def safe_process_row(row, row_index):
    """Process single row dengan concurrency control dan validation - 1 row = 1 JSONL."""
    async with SEM:
        # Ekstrak data untuk menentukan mode
        judul_tugas_akhir = normalize(row.get("Judul_Tugas_Akhir", ""))
        sertifikasi = normalize(row.get("Sertifikasi", ""))
        posisi_pekerjaan = normalize(row.get("Posisi_Pekerjaan", ""))
        level_okupasi = normalize(row.get("Level_Okupasi", ""))
        jenjang_pendidikan = normalize(row.get("Jenjang_Pendidikan", ""))
        
        # Cek kondisi untuk medium/long (data tidak lengkap)
        is_incomplete = (
            judul_tugas_akhir and "tidak ada tugas akhir" in judul_tugas_akhir.lower() or
            sertifikasi and "belum memiliki sertifikasi" in sertifikasi.lower() or
            posisi_pekerjaan and "belum memiliki pengalaman kerja" in posisi_pekerjaan.lower()
        )
        
        # Cek kondisi khusus: Level 9 + S3 = fast_direct
        is_level_9_s3 = False
        if level_okupasi and jenjang_pendidikan:
            try:
                level_val = str(int(float(level_okupasi)))
                is_level_9_s3 = (level_val == "9" and "s3" in jenjang_pendidikan.lower())
            except:
                pass
        
        # Tentukan mode berdasarkan kondisi
        if is_level_9_s3:
            mode = "fast_direct"
        elif is_incomplete:
            # Data tidak lengkap -> medium atau long
            mode = random.choice(["medium", "long"])
        else:
            # Data lengkap -> medium atau fast_direct
            mode = random.choice(["medium", "fast_direct"])
        
        prompt = build_prompt(row, mode)
        
        # Call API sekali saja
        try:
            result = await call_api(prompt, row_index, mode)
            return result
        except Exception as e:
            print(f"Exception (row {row_index}, {mode}): {str(e)}")
            return [
                {
                    "role": "user",
                    "content": "Berikut data singkat saya:\n(terjadi error saat memproses)."
                },
                {
                    "role": "assistant",
                    "content": f"[END OF CHAT] Terima kasih telah melakukan interview. Namun terjadi error: {str(e)} <RESULT>{{\"area_fungsi\":\"unknown\", \"level\":null}}</RESULT>"
                },
            ]

async def process_batch(df, writer, batch_num, pbar):
    """Process batch of rows dengan validation - 1 row = 1 JSONL."""
    tasks = []
    for idx, row in df.iterrows():
        tasks.append(safe_process_row(row, idx))
    
    results = []
    for coro in asyncio.as_completed(tasks):
        out = await coro
        results.append(out)
        pbar.update(1)
    
    # Write results dengan final validation
    valid_count = 0
    error_count = 0
    
    for msgs in results:
        # Final check before writing
        is_valid, _ = validate_conversation_structure(msgs)
        
        if is_valid:
            # Format output dengan wrapper "messages" dan system di dalamnya
            formatted_messages = [
                {"role": "system", "content": SYSTEM_PROMPT}
            ] + msgs
            
            obj = {
                "messages": formatted_messages
            }
            writer.write(json.dumps(obj, ensure_ascii=False) + "\n")
            valid_count += 1
        else:
            # Skip corrupt conversations
            error_count += 1
            print(f"Skipped corrupt conversation")
    
    if error_count > 0:
        print(f"Batch {batch_num}: {valid_count} valid, {error_count} skipped")

# PROCESS SINGLE FILE
async def process_single_file(input_path, output_dir, file_pbar=None):
    """Process satu file Excel."""
    file_name = Path(input_path).stem
    
    print(f"\n{'='*60}")
    print(f"Processing: {file_name}")
    print(f"{'='*60}")
    
    # Read Excel with error handling
    try:
        df = pd.read_excel(input_path)
        print(f"Columns detected: {df.columns.tolist()}")
    except FileNotFoundError:
        print(f"Error: File tidak ditemukan: {input_path}")
        if file_pbar:
            file_pbar.update(1)
        return
    except Exception as e:
        print(f"Error membaca Excel: {e}")
        if file_pbar:
            file_pbar.update(1)
        return
    
    total = len(df)
    print(f"Total rows: {total}")
    
    # Create output directory for this file
    file_output_dir = output_dir / file_name
    os.makedirs(file_output_dir, exist_ok=True)
    
    batch_count = 1
    
    # Progress bar for rows
    with tqdm(total=total, desc=f"  Rows ({file_name})", unit="row", leave=False) as pbar:
        for i in range(0, total, BATCH_SIZE):
            batch = df.iloc[i:i + BATCH_SIZE]
            
            file_path = f"{file_output_dir}/batch_{batch_count:03d}.jsonl"
            
            try:
                with open(file_path, "w", encoding="utf-8") as w:
                    await process_batch(batch, w, batch_count, pbar)
            except Exception as e:
                print(f"Error writing batch {batch_count}: {e}")
            
            batch_count += 1
    
    print(f"Done: {file_name} - {batch_count - 1} batches created")
    
    if file_pbar:
        file_pbar.update(1)

# MAIN - PROCESS ALL FILES
async def process_all_files(input_dir, output_base):
    """Process semua file Excel di direktori input."""
    print(f"\n{'='*60}")
    print(f"Multi-File Dataset Generation")
    print(f"{'='*60}")
    print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Input Dir: {input_dir}")
    print(f"Output Dir: {output_base}")
    print(f"Model: {MODEL_NAME}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Concurrent requests: {CONCURRENT_REQUESTS}")
    print(f"{'='*60}\n")
    
    # Find all Excel files
    excel_files = list(Path(input_dir).glob("*.xlsx"))
    
    if not excel_files:
        print("Tidak ada file Excel ditemukan!")
        return
    
    print(f"Found {len(excel_files)} Excel files\n")
    
    # Create base output directory
    os.makedirs(output_base, exist_ok=True)
    
    # Process files sequentially (untuk menghindari rate limit)
    with tqdm(total=len(excel_files), desc="Files", unit="file") as file_pbar:
        for excel_file in excel_files:
            await process_single_file(excel_file, output_base, file_pbar)
    
    print(f"\n{'='*60}")
    print(f"ALL DONE!")
    print(f"Output: {output_base}")
    print(f"Processed: {len(excel_files)} files")
    print(f"{'='*60}\n")

input_directory = DATASET_DIR
output_directory = OUTPUT_BASE_DIR
print("\nMemulai proses generation...")
print("Harap tunggu, ini akan memakan waktu...\n")

await process_all_files(input_directory, output_directory)

/Users/irz/projects/Dataset SFT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


python-dotenv tidak terinstall, gunakan environment variable manual
API Key loaded from .env file manually
API Key: sk-or-v1-0db430b734e... (truncated for security)
Output akan disimpan di: /Users/irz/Downloads/dtp-data-pipeline-main/Pipeline Multiturn/MultiturnDatasetOutput

Memulai proses generation...
Harap tunggu, ini akan memakan waktu...


Multi-File Dataset Generation
Time: 2025-12-11 01:01:27
Input Dir: /Users/irz/Downloads/dtp-data-pipeline-main/Pipeline Multiturn/Flagged_500_Per_Class/Rows_1-100
Output Dir: /Users/irz/Downloads/dtp-data-pipeline-main/Pipeline Multiturn/MultiturnDatasetOutput
Model: google/gemini-2.5-flash
Batch size: 10
Concurrent requests: 5

Found 45 Excel files



Files:   0%|          | 0/45 [00:00<?, ?file/s]


Processing: Pengembangan_Produk_Digital_5
Columns detected: ['Jenjang_Pendidikan', 'Jurusan', 'Judul_Tugas_Akhir', 'Bidang_Pelatihan', 'Nama_Pelatihan', 'Sertifikasi', 'Bidang_Sertifikasi', 'Posisi_Pekerjaan', 'Deskripsi_tugas_dan_tanggung_jawab', 'Lama_Bekerja', 'Keterampilan', 'Area_Fungsi', 'Level_Okupasi']
Total rows: 1


Parse failed (row 0, long, attempt 1): JSON parse error: Unterminated string starting at: line 56 column 16 (char 5857)


...

Done: Keamanan_Informasi_dan_Siber_3 - 1 batches created

Processing: Tata_Kelola_Teknologi_Informasi_5
Columns detected: ['Jenjang_Pendidikan', 'Jurusan', 'Judul_Tugas_Akhir', 'Bidang_Pelatihan', 'Nama_Pelatihan', 'Sertifikasi', 'Bidang_Sertifikasi', 'Posisi_Pekerjaan', 'Deskripsi_tugas_dan_tanggung_jawab', 'Lama_Bekerja', 'Keterampilan', 'Area_Fungsi', 'Level_Okupasi']
Total rows: 1


Files:  13%|█▎        | 6/45 [00:52<04:47,  7.38s/file]

Done: Tata_Kelola_Teknologi_Informasi_5 - 1 batches created

Processing: Layanan_Teknologi_Informasi_1
Columns detected: ['Jenjang_Pendidikan', 'Jurusan', 'Judul_Tugas_Akhir', 'Bidang_Pelatihan', 'Nama_Pelatihan', 'Sertifikasi', 'Bidang_Sertifikasi', 'Posisi_Pekerjaan', 'Deskripsi_tugas_dan_tanggung_jawab', 'Lama_Bekerja', 'Keterampilan', 'Area_Fungsi', 'Level_Okupasi']
Total rows: 1


Files:  13%|█▎        | 6/45 [00:54<05:53,  9.06s/file]


CancelledError: 